# DS 710 Final Project
## Name: You Li


In [1]:
##  Gain access to the Tweepy library
import tweepy

In [2]:
## Load your credentials from the external file
%run ~/twitter_credentials.py

In [3]:
#Use tweepy.OAuthHandler to create an authentication using the given key and secret
auth = tweepy.OAuthHandler(consumer_key=con_key, consumer_secret=con_secret)
auth.set_access_token(acc_token, acc_secret)

#Connect to the Twitter API using the authentication
api = tweepy.API(auth)

## PART A:  
### 1. Collect the 1000 tweets and find out the sentiment rate for each tweet.
### 2. Count words in positive tweets, and find out the top 10 meaningful words 


In [4]:
## Collect 1000 arbitrary tweets based on the key word "NewYorkCity" and create a list for them

num_needed = 1000
NYC_tweet_list = []
last_id = -1 # id of last tweet seen
while len(NYC_tweet_list) < num_needed:
    try:
        NYC_tweets = api.search(q = '#%23NewYorkCity', count = 100, max_id = str(last_id - 1))
    except tweepy.TweepError as e:
        print("Error", e)
        break
    else:
        if not NYC_tweets:
            print("Could not find any more tweets!")
            break
        NYC_tweet_list.extend(NYC_tweets)
        last_id = NYC_tweets[-1].id
len(NYC_tweet_list)        

1061

In [5]:
## Convert tweepy.models.Status to json file
import json
dict_list=[]
for json_tweet in NYC_tweet_list:
    dict_list.append(json_tweet._json)
with open('NYC_tweets.txt',"w")as file:
    file.write(json.dumps(dict_list,indent=4))

In [6]:
## create a list to collect the sentiment rate of each tweet.
sentiment_rate=[]
## create a list to collect words from tweets which have positive sentiment rates
single_word=[]
## create a list to collect words which we want to count (we don't count the words which are only marks )
count_words=[]

## use function Counter to count words
from collections import Counter
## use function TextBlob to analysis tweets' sentiment
from textblob import TextBlob

with open('NYC_tweets.txt', 'r') as t:
    json_list = json.load(t)
for i in range(len(json_list)):
    tweet_text_NYC = TextBlob(json_list[i]["text"])
    sentiment_rate.append(tweet_text_NYC.sentiment[0])
    if tweet_text_NYC .sentiment[0]>0:
        tweet_text_NYC =tweet_text_NYC.replace("RT","") 
        single_word=tweet_text_NYC.split()
        for ii in single_word: 
            if ii.isalnum()==True:
                count_words.append(ii)


In [7]:
## Some words are meaningless, such as preposition words, conjunction words, article words. Remove them.

## use nltk to identify words' categories
import nltk
## create a list to collect the meaningful words we want
meaningful_words=[]
pos_tags =nltk.pos_tag(count_words)
for tags in pos_tags:
    if tags[1]!= 'CC' and tags[1] !='IN' and tags[1]!='DT'\
    and tags[1]!= 'TO' and tags[1] != 'MD':    ## CC: coordinatingconjunction / IN :preposition/subordinating conjunction / TO :to/ MD: modal (could, will) / DT: determiner
        meaningful_words.append(tags)

## select top 10 meaningful words to make a data frame 
meaningful_words_nyc=Counter(meaningful_words).most_common(11)
print(meaningful_words_nyc)

[(('New', 'NNP'), 112), (('you', 'PRP'), 88), (('York', 'NNP'), 74), (('your', 'PRP$'), 71), (('my', 'PRP$'), 65), (('I', 'PRP'), 64), (('Soul', 'NNP'), 63), (('Day', 'NNP'), 60), (('Mothers', 'NNP'), 58), (('is', 'VBZ'), 48), (('Say', 'NNP'), 47)]


In [8]:
## Create a dataframe for the top 10 words with three columns "count","word","category"

import pandas as pd
NYC_TOP10=pd.DataFrame(meaningful_words_nyc)
def function(row):    
    return row[0][0]
NYC_TOP10["word"]=NYC_TOP10.apply(function,axis=1)
def function2(row):    
    return row[0][1]
NYC_TOP10["category"]=NYC_TOP10.apply(function2,axis=1)
NYC_TOP10=NYC_TOP10.drop([0], axis=1)
NYC_TOP10.rename(columns={ NYC_TOP10.columns[0]: "count" }, inplace=True)
print(NYC_TOP10)
NYC_TOP10.to_csv("NYC_TOP10.csv",index=False,sep=',')     ## Save the dataframe as csv file for R analysis

    count     word category
0     112      New      NNP
1      88      you      PRP
2      74     York      NNP
3      71     your     PRP$
4      65       my     PRP$
5      64        I      PRP
6      63     Soul      NNP
7      60      Day      NNP
8      58  Mothers      NNP
9      48       is      VBZ
10     47      Say      NNP


In [9]:
## Do the same five steps with the key word "LosAngles" 
## We will get a top 10 words dataframe from LA related tweets

num_needed = 1000
LA_tweet_list = []
last_id = -1 # id of last tweet seen
while len(LA_tweet_list) < num_needed:
    try:
        LA_tweets = api.search(q = '#%23LosAngeles', count = 100, max_id = str(last_id - 1))
    except tweepy.TweepError as e:
        print("Error", e)
        break
    else:
        if not LA_tweets:
            print("Could not find any more tweets!")
            break
        LA_tweet_list.extend(LA_tweets)
        last_id = LA_tweets[-1].id

len(LA_tweet_list)

1018

In [10]:
import json
dict_list2=[]
for json_tweet in LA_tweet_list:
    dict_list2.append(json_tweet._json)
with open('LA_tweets.txt',"w")as file:
    file.write(json.dumps(dict_list2,indent=4))

In [11]:
sentiment_rate_LA=[]
single_word_LA=[]
count_words_LA=[]

with open('LA_tweets.txt', 'r') as t2:
    json_list2 = json.load(t2)
for k in range(len(json_list2)):
    tweet_text_LA = TextBlob(json_list2[k]["text"])
    sentiment_rate_LA.append(tweet_text_LA.sentiment[0])
    if tweet_text_LA.sentiment[0]>0:
        tweet_text_LA=tweet_text_LA.replace("RT","")
        single_word_LA=tweet_text_LA.split()
        for kk in single_word_LA:
            if kk.isalnum()==True:
                count_words_LA.append(kk)


In [12]:
meaningful_words_LA=[]
pos_tags =nltk.pos_tag(count_words_LA)
for tags in pos_tags:
    if tags[1]!= 'CC' and tags[1] !='IN' and tags[1]!='DT' and tags[1]!= 'TO' and tags[1] != 'MD':
        meaningful_words_LA.append(tags)
    
meaningful_words_LA=Counter(meaningful_words_LA).most_common(11)
meaningful_words_LA


[(('is', 'VBZ'), 60),
 (('you', 'PRP'), 53),
 (('our', 'PRP$'), 47),
 (('I', 'PRP'), 46),
 (('be', 'VB'), 34),
 (('your', 'PRP$'), 31),
 (('my', 'PRP$'), 28),
 (('job', 'NN'), 28),
 (('are', 'VBP'), 25),
 (('new', 'JJ'), 20),
 (('great', 'JJ'), 19)]

In [13]:
import pandas as pd
LA_TOP10=pd.DataFrame(meaningful_words_LA)
def function(row):    
    return row[0][0]
LA_TOP10["word"]=LA_TOP10.apply(function,axis=1)
def function2(row):    
    return row[0][1]
LA_TOP10["category"]=LA_TOP10.apply(function2,axis=1)
LA_TOP10=LA_TOP10.drop([0], axis=1)
LA_TOP10.rename(columns={ LA_TOP10.columns[0]: "count" }, inplace=True)
print(LA_TOP10)
LA_TOP10.to_csv("LA_TOP10.csv",index=False,sep=',')

    count   word category
0      60     is      VBZ
1      53    you      PRP
2      47    our     PRP$
3      46      I      PRP
4      34     be       VB
5      31   your     PRP$
6      28     my     PRP$
7      28    job       NN
8      25    are      VBP
9      20    new       JJ
10     19  great       JJ


In [14]:
## Create a dataframe for 1000 sentiment rates of 1000 NYC related tweets and LA related tweets

NYC_VS_LA = pd.DataFrame({'NYC':sentiment_rate[0:1000],'LA':sentiment_rate_LA[0:1000]})
NYC_VS_LA [0:20]

,NYC,LA
0,0.539205,-0.097222
1,0.800000,0.600000
2,0.775000,0.800000
3,0.500000,0.136364
4,0.000000,0.000000
5,0.000000,0.000000
6,0.000000,0.000000
7,0.000000,0.000000
8,0.000000,-0.400000
9,0.800000,1.000000


In [15]:
## Save the dataframe as csv file for R analysis
NYC_VS_LA.to_csv("NYC_VS_LA.csv",index=False,sep=',')

## PART B:
### Use the attribute "place" of tweet to distinguish "residents" and "tourists"
### Find out the sentiment rate for the placed tweets

In [16]:
## Create a list to collect NYC related tweets which has been placed
NYC_placed_tweets=[]
## Create a list to collect the sentiment rate of NYC placed tweets
NYC_placed_tweets_sentiment=[]
for l in range(len(json_list)): 
    if json_list[l]['place'] != None:
        NYC_placed_tweets.append(json_list[l]['place']['full_name'])
        tweet_text_NYC = TextBlob(json_list[l]["text"])
        NYC_placed_tweets_sentiment.append(tweet_text_NYC.sentiment[0])
## check the length of the list
len(NYC_placed_tweets)

121

In [17]:
## Create a list to collect LA related tweets which has been placed
LA_placed_tweets=[]
## Create a list to collect the sentiment rate of LA placed tweets
LA_placed_tweets_sentiment=[]
with open('LA_tweets.txt', 'r') as tt:
    json_list2 = json.load(tt)
for n in range(len(json_list2)): 
    if json_list2[n]['place'] != None:
        LA_placed_tweets.append(json_list2[n]['place']['full_name'])
        tweet_text_LA = TextBlob(json_list2[n]["text"])
        LA_placed_tweets_sentiment.append(tweet_text_LA .sentiment[0])
len(LA_placed_tweets)

181

In [18]:
## Create a dataframe for above four lists

import pandas as pd
dataframe_place = pd.DataFrame({'NYC_placed_tweets':NYC_placed_tweets[0:120],"NYC_placed_tweets_sentiment":NYC_placed_tweets_sentiment[0:120],'LA_placed_tweets':LA_placed_tweets[0:120],'LA_placed_tweets_sentiment':LA_placed_tweets_sentiment[0:120]})
dataframe_place

,NYC_placed_tweets,NYC_placed_tweets_sentiment,LA_placed_tweets,LA_placed_tweets_sentiment
0,"Bronx, NY",0.350000,"Los Angeles, CA",0.600000
1,"Brooklyn, NY",0.247222,"Los Angeles, CA",0.136364
2,"Manhattan, NY",0.200000,"San Clemente, CA",1.000000
3,"Manhattan, NY",-0.075000,"Los Angeles, CA",0.000000
4,"Manhattan, NY",0.000000,"Los Angeles, CA",0.233333
5,"Nantucket, MA",0.000000,"Torrance, CA",0.000000
6,"Manhattan, NY",0.000000,"Los Angeles, CA",0.000000
7,"Manhattan, NY",-0.166667,"Los Angeles, CA",0.000000
8,"Manhattan, NY",0.136364,"Los Angeles, CA",1.000000
9,United States,0.000000,"West Hollywood, CA",0.000000


In [19]:
dataframe_place.to_csv("place_sentiment rate.csv",index=False,sep=',')